#**Import Statements + Global Functions**

In [1]:
import numpy as np
import numpy.random as rand

In [38]:
class Cell():
    target = 0
    cell_type = ""
    prob_not_found = 0
    def __init__(self, cell_type, prob_not_found):
        self.cell_type = cell_type
        self.prob_not_found = prob_not_found
        

In [97]:
#create environment
#take input from user
while True:
  try:
    dimension = int(input("Enter dimension of board: "))
    break
  except ValueError:
    print("Must be an integer")
    
env = [[] for i in range(dimension)] # create a list of empty lists
cell_type = ""
prob_not_found = 0
for i in range(dimension):
    for j in range(dimension):
        p = rand.random()
        if p < 0.25:
            cell_type = "flat"
            prob_not_found = 0.1
        elif p <0.50:
            cell_type = "hilly"
            prob_not_found = 0.3
        elif p <0.75:
            cell_type = "forested"
            prob_not_found = 0.7
        else:
            cell_type = "maze of caves"
            prob_not_found = 0.9
        env[i].append(Cell(cell_type, prob_not_found))

Enter dimension of board: 5


In [91]:
env[0][0].cell_type = "flat"
env[0][0].prob_not_found = 0.1
env[0][1].cell_type = "hilly"
env[0][1].prob_not_found = 0.3
env[1][0].cell_type = "forested"
env[1][0].prob_not_found = 0.7
env[1][0].target = 1
env[1][1].cell_type = "maze of caves"
env[1][1].prob_not_found = 0.9

In [98]:
env[1][0].target = 1

In [99]:
print(env[1][0].target)

1


In [69]:
def search_cell(current_position):
    x = current_position[0]
    y = current_position[1]
    
    if env[x][y].target == 1:
        p = rand.random()
        if p > env[x][y].prob_not_found:
            return True
    return False

# **Basic Agent 1**

In [44]:
#variable assignment tree
def bayes_theorem(current_position, updating_position):
    x = current_position[0]
    y = current_position[1]
    i = updating_position[0]
    j = updating_position[1]
    
    numerator = belief[i][j]
    #print(f"numerator: {numerator}")
    denominator = (1 - belief[x][y]) + belief[x][y]*env[x][y].prob_not_found
    #print(f"denominator: {denominator}")
    return numerator/denominator 

In [45]:
def bayes_current(current_position):
    x = current_position[0]
    y = current_position[1]
    
    numerator = belief[x][y] * env[x][y].prob_not_found
    denominator = (1 - belief[x][y]) + belief[x][y]*env[x][y].prob_not_found
    return numerator/denominator 

In [46]:
def get_valid_neighbors(current_node):
  get_neighbors = [] #list of possible neighbors
  for i in range(-1,2):
    for j in range(-1,2):
      if i==0 and j==0:
        continue
      get_neighbors.append((i,j))
  x, y = current_node
  neighbors = set([])
  for i in get_neighbors:
    neighbor_x = x + i[0]
    neighbor_y = y + i[1]

    if neighbor_x >= dimension or neighbor_x < 0:
      continue
    elif neighbor_y >= dimension or neighbor_y < 0:
      continue
    neighbors.add((neighbor_x, neighbor_y))
  return neighbors

In [76]:
belief = np.zeros((dimension,dimension))
for i in range(dimension):
    for j in range(dimension):
        belief[i][j] = 1/(dimension*dimension)
print(belief)
searching = True
current_position = [0,0]
while searching:
    print(f"current_position: {current_position}")
    #search cell
    query = search_cell(current_position)
    if query:
        print(f"Found target at {current_position}")
        searching = False
        break
        
    #update belief
    for i in range(dimension):
        for j in range(dimension):
            if i == current_position[0] and j == current_position[1]:
                continue
            #Bayes Theorem
            calculated_belief = bayes_theorem(current_position, (i,j))
            belief[i][j] = calculated_belief #update belief
    belief[current_position[0]][current_position[1]] = bayes_current(current_position)
    print(belief)
    
    #choose next position
    max_belief = 0
    total_belief = belief[current_position[0]][current_position[1]]
    for i in get_valid_neighbors(current_position):
        total_belief += belief[i[0]][i[1]]
        if belief[i[0]][i[1]] > max_belief:
            max_belief = belief[i[0]][i[1]]
            current_position = i

    #print(f"sum of belief: {total_belief}")
    print(f"sum of belief: {sum(sum(belief))}")
    print()

[[0.25 0.25]
 [0.25 0.25]]
current_position: [0, 0]
[[0.18918919 0.27027027]
 [0.27027027 0.27027027]]
sum of belief: 0.9999999999999999

current_position: (0, 1)
[[0.20588235 0.20588235]
 [0.29411765 0.29411765]]
sum of belief: 0.9999999999999998

current_position: (1, 0)
Found target at (1, 0)


# **Basic Agent 2**

In [100]:
belief = np.zeros((dimension,dimension))
for i in range(dimension):
    for j in range(dimension):
        belief[i][j] = 1/(dimension*dimension)
print(belief)
searching = True
current_position = [0,0]
while searching:
    print(f"current_position: {current_position}")
    #search cell
    query = search_cell(current_position)
    if query:
        print(f"Found target at {current_position}")
        searching = False
        break
        
    #update belief
    for i in range(dimension):
        for j in range(dimension):
            if i == current_position[0] and j == current_position[1]:
                continue
            #Bayes Theorem
            calculated_belief = bayes_theorem(current_position, (i,j))
            belief[i][j] = calculated_belief #update belief
    belief[current_position[0]][current_position[1]] = bayes_current(current_position)
    print(belief)
    
    #choose next position
    max_belief = 0
    total_belief = belief[current_position[0]][current_position[1]]*(1-env[current_position[0]][current_position[1]].prob_not_found)
    for i in get_valid_neighbors(current_position):
        total_belief += belief[i[0]][i[1]]*(1 - env[i[0]][i[1]].prob_not_found)
        if belief[i[0]][i[1]]*(1-env[i[0]][i[1]].prob_not_found) > max_belief:
            max_belief = belief[i[0]][i[1]]
            current_position = i
    #print(f"sum of belief: {total_belief}")
    print(f"sum of belief: {sum(sum(belief))}")
    print()

[[0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]]
current_position: [0, 0]
[[0.01234568 0.04115226 0.04115226 0.04115226 0.04115226]
 [0.04115226 0.04115226 0.04115226 0.04115226 0.04115226]
 [0.04115226 0.04115226 0.04115226 0.04115226 0.04115226]
 [0.04115226 0.04115226 0.04115226 0.04115226 0.04115226]
 [0.04115226 0.04115226 0.04115226 0.04115226 0.04115226]]
sum of belief: 1.0

current_position: (0, 1)
[[0.01282051 0.0042735  0.04273504 0.04273504 0.04273504]
 [0.04273504 0.04273504 0.04273504 0.04273504 0.04273504]
 [0.04273504 0.04273504 0.04273504 0.04273504 0.04273504]
 [0.04273504 0.04273504 0.04273504 0.04273504 0.04273504]
 [0.04273504 0.04273504 0.04273504 0.04273504 0.04273504]]
sum of belief: 1.0

current_position: (1, 2)
[[0.01333333 0.00444444 0.04444444 0.04444444 0.04444444]
 [0.04444444 0.04444444 0.00444444 0.04444444 0.04444444]
 [0.04444444 0.04444444 0.04444444 0.04444444 

# **Improved Agent**

In [ ]:
belief = np.zeros((dimension,dimension))
for i in range(dimension):
    for j in range(dimension):
        belief[i][j] = env[current_position[0]][current_position[1]].prob_not_found/(dimension*dimension)
print(belief)
searching = True
current_position = [0,0]
while searching:
    print(f"current_position: {current_position}")
    #search cell
    query = search_cell(current_position)
    if query:
        print(f"Found target at {current_position}")
        searching = False
        break
        
    #update belief
    for i in range(dimension):
        for j in range(dimension):
            if i == current_position[0] and j == current_position[1]:
                continue
            #Bayes Theorem
            calculated_belief = bayes_theorem(current_position, (i,j))
            belief[i][j] = calculated_belief #update belief
    belief[current_position[0]][current_position[1]] = bayes_current(current_position) * env[current_position[0]][current_position[1]].prob_not_found
    print(belief)
    
    #choose next position
    max_belief = 0
    total_belief = belief[current_position[0]][current_position[1]]
    for i in get_valid_neighbors(current_position):
        total_belief += belief[i[0]][i[1]]
        if belief[i[0]][i[1]] > max_belief:
            max_belief = belief[i[0]][i[1]]
            current_position = i
    print(f"sum of belief: {total_belief}")